In [23]:
import numpy as np
import hashlib
import time
import math

#バイナリ法
def mod_binary(g, k, p):
     bk = bin(k)[2:]
     y = 1
     for i in bk:
        y = pow(y, 2, p)
        if i == '1':
            y = (y*g) % p
     return y

#ユークリッドの互除法
def euclid(a, b):
    r = a % b
    while r != 0:
        a, b = b, r
        r = a % b
    return b

def ex_euclid(a, b):
    x0, x1 = 1, 0
    y0, y1 = 0, 1
    while b != 0:
        q, r = divmod(a, b)
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
        a, b = b, r
    return [x0, y0, a]

#逆元計算
def inv(a, n):
    inv_x0, inv_y0, b= ex_euclid(a, n)
    return inv_x0 % n

#最小公倍数
def lcm(a, b):
  return a*b/euclid(a, b)

#rsa暗号-鍵生成
def rsaKeygen(p, q):
  lam = lcm(p-1, q-1)
  e = Fermat(10, 1024, 1)[0]
  n = p * q
  d = inv(e, lam)
  return [e, n, d]

#rsa暗号-暗号化
def rsaEnc(m, n, e):#m(メッセージ):整数, #(n, e):公開鍵
  c = mod_binary(m, e, n)
  return c

#rsa暗号-復号
def rsaDec(c, n, d):
  m = mod_binary(c, d, n)
  return m

#RSA署名鍵生成
def rsaSignGenKey(p, q, e):
  n = p * q
  d = inv(e, lcm(p-1, q-1))
  return [n, e], d

#RSA署名生成関数
def rsaSignGen(m, d, n):
  return mod_binary(shake128(m, 127), d, n)

#RSA署名検証関数
def rsaSignVerify(m, sigma, e, n):
  return shake128(m, 127) == mod_binary(sigma, e, n)

def str_to_int(m):
    m_b = m.encode('cp932')
    m_int = int.from_bytes(m_b, "big")
    return m_int

def int_to_str(m):
  length = math.ceil(int.bit_length(m)/8)
  m_b = m.to_bytes(length, "big")
  m_str = m_b.decode('cp932')
  return m_str

# Input: メッセージ: m (文字列), ハッシュ値のサイズ: h_size (バイト)
# Output: m のshake128によるハッシュ値（整数値）
def shake128(m, h_size):
    mh = hashlib.shake_128(m.encode()).hexdigest(h_size)
    return int(mh, 16)



In [24]:
m32 = '大変興味深かった.'
m32int = str_to_int(m32)
#特に記載がなかったので(n, d)は表D.9の値を使用しました.
d = 131443304793927428151332221511108704880067947255385616574001106543358350512695028610591682664665998149759408723503668627360294194220319028140588345821358616727028325663441231913951992382845289448473974065202883492432548694746970307278305986911096630577768359426701452770550634580605686885500693045782534383513
n =  151688675229435144545762630765513843840905318881426433410993317829407927761058180842619248226698635688339185939641837133849473509537190493876558169045507678528180233325144584382935801344398779896870342021499422411253803876549156126455881007490336116762896649111675339984255409282034328119927716206914899875961
#(n, e)
pk_kawahara = [69179065868980767651505682212128103059156904708577280552708201801233407073165594460573526256104767274049921556957457159754595905781553144753550859791391645577404627016846972912032792419231368409024280895909536375867295542657404819386276005111369198385611268710168382234914840652752969638548435338458341587443,65537]
enc_m32 = rsaEnc(m32int, pk_kawahara[0], pk_kawahara[1])
print('暗号化結果：{}'.format(enc_m32))
sign_m32 = rsaSignGen(m32, d, n)
print("署名結果：{}".format(sign_m32))

暗号化結果：27659542603684648283959164811331828178601643720953424297316861167846127063075439085360665523843957554821252613403839509180182915076600117539083678911048136089115216887083307963928653926862600834949879031236660644703267170736109179614139247921948758281694517640006863801351608959024536700985088839464839867144
署名結果：90001397255603635812488292559780329803653271990009420254489329205107543976763363478875321654797253455832637435537336323934575631714599152132624694595439721399189806985377751142975249098005161223287122848827154402826264366249495431783082965855953561718875792295805084863760660876501659958573541942431165401681


In [25]:
m_dec = rsaDec(enc_m32, n, d)
m = int_to_str(m_dec)
# dec_m = rsaDec(enc_m32,n,d)
# dec_m = dec_m.to_bytes((dec_m.bit_length() + 7) // 8, byteorder='big')
# m = dec_m.decode()

UnicodeDecodeError: 'cp932' codec can't decode byte 0xec in position 7: illegal multibyte sequence